In [1]:
import os

from datasets import load_dataset, load_from_disk

# Paths
cache_dir = "./cache"
processed_path = "./processed/alpaca-ptbr"

# Check if the processed dataset already exists
if os.path.exists(processed_path):
    print("📂 Loading dataset from local disk...")
    ds = load_from_disk(processed_path)
else:
    print("⬇️ Downloading and caching dataset...")
    ds = load_dataset("dominguesm/alpaca-data-pt-br", cache_dir=cache_dir)
    print("💾 Saving processed dataset to disk...")
    ds["train"].save_to_disk(processed_path)

# ds is ready to use
print(ds)

📂 Loading dataset from local disk...
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 51759
})


In [2]:
from transformers import AutoTokenizer

# Example usage of the GPT model
tokenizer = AutoTokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")

In [3]:
try:
    df = ds["train"].to_pandas()
except:
    df = ds.to_pandas()

In [4]:
df["concat"] = (
    df["instruction"] + "\n\n" + df["input"].fillna("") + "\n\n" + df["output"]
)

In [5]:
# create a column with count of tokens for each row
df["tokens"] = df["concat"].apply(lambda x: len(tokenizer.encode(x)))

In [6]:
df["tokens"].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

count    51759.000000
mean        74.283371
std         40.517703
min         10.000000
25%         38.000000
50%         67.000000
75%        110.000000
90%        127.000000
95%        136.000000
99%        159.000000
max        838.000000
Name: tokens, dtype: float64

## Train

In [7]:
import torch

from model import GPT, GPTConfig

device = "cuda" if torch.cuda.is_available() else "cpu"

n_experts = 4
factor = 2
config = GPTConfig(
    block_size=128,
    vocab_size=tokenizer.vocab_size,
    n_layer=2 * factor,
    n_head=4 * factor,
    n_embd=256 * factor,
    n_experts=n_experts,
    capacity_factor=n_experts,
    k=2 * factor,
    experts_weight=1e-5,
    router_weight=1e-5,
    dropout=0.2,
    bias=True,
)

model = GPT(config)

number of parameters: 64.60M


In [8]:
inputs = tokenizer(
    df["concat"].tolist(),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=128,
)

input_ids = inputs["input_ids"]
targets = input_ids.clone()

targets[:, :-1] = input_ids[:, 1:]
targets[:, -1] = -1

# convert targets 0 to -1
targets[targets == 0] = -1

# print("input ids:", inputs["input_ids"], "shape:", inputs["input_ids"].shape)

In [9]:
input_ids.shape

torch.Size([51759, 128])

In [10]:
# invert 0 -> 1 and 1 -> 0 in attention mask
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# create a new upper triangular mask
upper_mask = torch.triu(torch.ones(input_ids.shape[1], input_ids.shape[1]), diagonal=1)

# apply the mask to the attention mask
attention_mask = attention_mask.unsqueeze(1) * upper_mask.unsqueeze(0)

# conver to bool
attention_mask = attention_mask.bool()

In [11]:
input_ids.shape

torch.Size([51759, 128])

In [12]:
model(input_ids[:5, :], attention_mask=attention_mask[:5, :, :])[0].shape

Load: tensor([1., 1., 1., 1.]), Importance: tensor([0.2518, 0.2421, 0.2503, 0.2558], grad_fn=<MeanBackward1>)
Router loss: 1.7503667550045066e-05, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.]), Importance: tensor([0.2229, 0.2183, 0.2526, 0.3062], grad_fn=<MeanBackward1>)
Router loss: 2.5536155590089038e-05, Balance loss: 9.99999883788405e-06
Load: tensor([1., 1., 1., 1.]), Importance: tensor([0.1974, 0.3003, 0.3191, 0.1831], grad_fn=<MeanBackward1>)
Router loss: 2.4078175556496717e-05, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.]), Importance: tensor([0.2711, 0.2988, 0.1397, 0.2904], grad_fn=<MeanBackward1>)
Router loss: 2.3265763957169838e-05, Balance loss: 1.0000000656873453e-05


torch.Size([5, 1, 50257])

### Train loop

In [13]:
# send data to device
model = model.to(device)
input_ids = input_ids[:32, :].to(device)  # limit to 16 for testing
attention_mask = attention_mask[:32, :, :].to(device)
targets = targets[:32, :].to(device)

# get only one sample for each
input_ids = input_ids[:64, :]
attention_mask = attention_mask[:64, :, :]
targets = targets[:64, :]

In [14]:
input_ids.shape

torch.Size([32, 128])

In [15]:
from tqdm.notebook import tqdm

with torch.no_grad():
    logits, loss = model(input_ids, attention_mask, targets)
    # print("logits:", logits, "shape:", logits.shape)
print("loss:", loss.item() if loss is not None else "N/A")
if loss is not None:
    perplexity = torch.exp(loss)
    print("perplexity:", perplexity.item())

# pred_tokens = logits.argmax(dim=-1)
# tokens = pred_tokens[0].tolist()
# print("predicted tokens:", tokens)
# print("predicted text:", tokenizer.decode(tokens))

optimizer = model.configure_optimizers(
    weight_decay=0.0,
    learning_rate=3e-4,
    betas=(0.9, 0.95),
    device_type="cuda",
)

pbar = tqdm(range(450), desc="Training Epochs")

for epoch in pbar:
    model.train()
    optimizer.zero_grad()
    logits, loss = model(input_ids, attention_mask, targets)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        logits, loss = model(input_ids, attention_mask, targets)

    perplexity = torch.exp(loss)
    pbar.set_postfix(loss=loss.item(), perplexity=perplexity.item())

    pred_tokens = logits.argmax(dim=-1)
    tokens = pred_tokens[0].tolist()

# Router loss: 0.0004672443028539419, Balance loss: 0.019999997690320015

Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.2635, 0.2291, 0.2507, 0.2568], device='cuda:0')
Router loss: 1.8157268641516566e-05, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.2193, 0.2250, 0.2480, 0.3076], device='cuda:0')
Router loss: 2.561533074185718e-05, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.2011, 0.3103, 0.3012, 0.1874], device='cuda:0')
Router loss: 2.4392165869357996e-05, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.2746, 0.2858, 0.1521, 0.2875], device='cuda:0')
Router loss: 2.253517959616147e-05, Balance loss: 9.999999747378752e-06
loss: 10.914222717285156
perplexity: 54952.40625
using fused AdamW: True


Training Epochs:   0%|          | 0/450 [00:00<?, ?it/s]

Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.2626, 0.2258, 0.2519, 0.2596], device='cuda:0',
       grad_fn=<MeanBackward1>)
Router loss: 1.8201668353867717e-05, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.2183, 0.2252, 0.2484, 0.3081], device='cuda:0',
       grad_fn=<MeanBackward1>)
Router loss: 2.5558778361300938e-05, Balance loss: 9.99999883788405e-06
Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.2045, 0.3070, 0.3036, 0.1849], device='cuda:0',
       grad_fn=<MeanBackward1>)
Router loss: 2.46496383624617e-05, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.2797, 0.2841, 0.1531, 0.2832], device='cuda:0',
       grad_fn=<MeanBackward1>)
Router loss: 2.255793515359983e-05, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.2593, 0.2334, 0.2509, 0.2564], device

In [16]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids[:, :1].clone(),
        max_new_tokens=100,
        temperature=0.1,
        top_k=1,
        greedy=True,
    )
generated_text = tokenizer.decode(generated_ids[0].tolist())
_df = df.head(32)
_df["generated_text"] = [
    tokenizer.decode(generated_ids[i].tolist()) for i in range(len(generated_ids))
]
print(f"Generated text after epoch {epoch + 1}: {generated_text}")


Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.1557, 0.0971, 0.6554, 0.0919], device='cuda:0')
Router loss: 1.3840906376572093e-06, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.0488, 0.1321, 0.1502, 0.6690], device='cuda:0')
Router loss: 5.044724389335897e-07, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.3269, 0.2485, 0.3038, 0.1209], device='cuda:0')
Router loss: 3.961779100336571e-07, Balance loss: 9.99999883788405e-06
Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.1987, 0.3327, 0.2437, 0.2249], device='cuda:0')
Router loss: 2.3827963957501197e-07, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.], device='cuda:0'), Importance: tensor([0.1438, 0.0849, 0.6870, 0.0844], device='cuda:0')
Router loss: 9.291302376368549e-07, Balance loss: 9.999999747378752e-06
Load: tensor([1., 1., 1., 1.], device='cuda:

C:\Users\mathe\AppData\Local\Temp\ipykernel_15556\3646376486.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df["generated_text"] = [


In [18]:
_df.head(32).style

,instruction,input,output,concat,tokens,generated_text
0,Dê três dicas para se manter saudável.,,1. Coma uma dieta equilibrada e certifique-se de incluir muitas frutas e vegetais. 2. Exercite-se regularmente para manter seu corpo ativo e forte. 3. Durma o suficiente e mantenha um horário de sono consistente.,Dê três dicas para se manter saudável. 1. Coma uma dieta equilibrada e certifique-se de incluir muitas frutas e vegetais. 2. Exercite-se regularmente para manter seu corpo ativo e forte. 3. Durma o suficiente e mantenha um horário de sono consistente.,62,"Dê três dicas para se manter saudável. 1. Coma uma dieta equilibrada e certifique-se de incluir muitas frutas e vegetais. 2. Exercite-se regularmente para manter seu corpo ativo e forte. 3. Durma o suficiente e mantenha um horário de sono consistente. utensílios de incluir muitas frutas e r é então dividida em moléculas mais simples de dióxido de empréstimos para financiar investimentos, superprodução agrícola e uma diminuição no comércio internacional. Exercite-se regularmente para"
1,Quais são as três cores primárias?,,"As três cores primárias são vermelho, azul e amarelo.","Quais são as três cores primárias? As três cores primárias são vermelho, azul e amarelo.",23,"Quais são as três cores primárias? As três cores primárias são vermelho, azul e amarelo. Tenda 2. Sacos de dormir 3. lanterna 4. fósforos / isqueiro 5. repelente de insetos 6. roupas extras 7. kit de primeiros socorros 8. Fogão do acampamento 9. utensílios de cozinha 10. sabonetes biodegradáveis / shampoos com números e gostava de energia liberada é usada para o ponto de trifosfato de adenos"
2,Descreva a estrutura de um átomo.,,"Um átomo é composto de um núcleo, que contém prótons e nêutrons, cercados por elétrons que viajam em órbitas ao redor do núcleo. Os prótons e nêutrons têm uma carga positiva, enquanto os elétrons têm uma carga negativa, resultando em um átomo neutro geral. O número de cada partícula determina o número atômico e o tipo de átomo.","Descreva a estrutura de um átomo. Um átomo é composto de um núcleo, que contém prótons e nêutrons, cercados por elétrons que viajam em órbitas ao redor do núcleo. Os prótons e nêutrons têm uma carga positiva, enquanto os elétrons têm uma carga negativa, resultando em um átomo neutro geral. O número de cada partícula determina o número atômico e o tipo de átomo.",78,"Descreva a vida e o reinado do rei Carlos II. O rei Carlos II ascendeu ao trono da Inglaterra após a Guerra Civil Inglesa e a morte de seu pai, Carlos I. Ele foi coroado em 1661 e seu reinado durou até sua morte em 1685. Carlos II também foi um patrono das artes, trabalhando com alguns dos maiores pintores e escultores de sua época. Ele também incentivou o teatro e os esportes e tinha interesse em ciências que também ajudaram a astronomia e a história natural. Carlos"
3,Como podemos reduzir a poluição do ar?,,"Há várias maneiras de reduzir a poluição do ar, como a mudança para fontes de energia renováveis, incentivando o uso de transporte público, proibindo a queima de combustíveis fósseis, implementando políticas para reduzir as emissões de fontes industriais e implementando padrões de emissões de veículos. Além disso, os indivíduos podem fazer sua parte para reduzir a poluição do ar, reduzindo o uso de carros, evitando a queima de materiais como madeira e mudando para aparelhos com eficiência energética.","Como podemos reduzir a poluição do ar? Há várias maneiras de reduzir a poluição do ar, como a mudança para fontes de energia renováveis, incentivando o uso de transporte público, proibindo a queima de combustíveis fósseis, implementando políticas para reduzir as emissões de fontes industriais e implementando padrões de emissões de veículos. Além disso, os indivíduos podem fazer sua parte para reduzir a poluição do ar, reduzindo o uso de carros, evitando a queima de materiais como madeira e mudando para aparelhos com eficiência energética.",101,"Como podemos reduzir a poluição do ar? Há vári